In [1]:
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score

In [2]:
# ==== 1 Exercício ====

# Criando o dataset

np.random.seed(42)
n = 100

dados = pd.DataFrame({
    "carga_ton": np.random.uniform(2, 15, n),
    "tempo_trajeto": np.random.uniform(10, 60, n),
    "velocidade_media": np.random.uniform(20, 80, n),
    "tipo_piso": np.random.choice(["asfalto", "brita", "terra"], n)
})

# Codificando a variável (tipo_piso)
dados = pd.get_dummies(dados, columns=["tipo_piso"], drop_first=False)

# Criando a variável alvo (consumo)
dados["consumo_litros"] = (
    0.8 * dados["carga_ton"] +
    0.05 * dados["tempo_trajeto"] -
    0.03 * dados["velocidade_media"] +
    3 * dados["tipo_piso_brita"] +
    5 * dados["tipo_piso_terra"] +
    np.random.normal(0, 1.5, n)
)

print("Amostra dos dados:")
print(dados.head())

Amostra dos dados:
   carga_ton  tempo_trajeto  velocidade_media  tipo_piso_asfalto  \
0   6.869022      11.571459         58.521899               True   
1  14.359286      41.820521         25.048398              False   
2  11.515921      25.717799         29.697723              False   
3   9.782560      35.428535         73.913251              False   
4   4.028242      55.378324         56.385744              False   

   tipo_piso_brita  tipo_piso_terra  consumo_litros  
0            False            False        3.690370  
1             True            False       15.659875  
2            False             True       12.292107  
3            False             True       14.055418  
4            False             True        7.566664  


In [3]:
# Separando variáveis de entrada (X) e saída (y)
X = dados.drop(columns="consumo_litros")
y = dados["consumo_litros"]

# Treinando e testando
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=42)

# Treinando dois modelos
modelos = {
    "Regressão Linear": LinearRegression(),
    "Random Forest": RandomForestRegressor(random_state=42, n_estimators=100)
}

# Avaliando o desempenho
resultados = []
for nome, modelo in modelos.items():
    modelo.fit(X_train, y_train)
    y_pred = modelo.predict(X_test)

    eqm = mean_squared_error(y_test, y_pred)
    eam = mean_absolute_error(y_test, y_pred)
    r2 = r2_score(y_test, y_pred)

    resultados.append({
        "Modelo": nome,
        "EQM": round(eqm, 2),
        "EAM": round(eam, 2),
        "R²": round(r2, 3)
    })

resultados_df = pd.DataFrame(resultados)
print("\n Resultados de desempenho dos modelos:")
print(resultados_df)


 Resultados de desempenho dos modelos:
             Modelo   EQM   EAM     R²
0  Regressão Linear  2.95  1.36  0.817
1     Random Forest  5.08  1.84  0.685


In [4]:
# Fazendo previsão para outro cenário
novo_dado = pd.DataFrame({
    "carga_ton": [10],
    "tempo_trajeto": [40],
    "velocidade_media": [55],
    "tipo_piso_asfalto": [0],
    "tipo_piso_brita": [1],
    "tipo_piso_terra": [0]
})

# Usando o modelo com melhor desempenho (ex: Random Forest)
melhor_modelo = RandomForestRegressor(random_state=42, n_estimators=100)
melhor_modelo.fit(X_train, y_train)
previsao = melhor_modelo.predict(novo_dado)

print(f"\n Previsão de consumo para o novo trajeto (brita): {previsao[0]:.2f} litros/hora")


 Previsão de consumo para o novo trajeto (brita): 12.18 litros/hora


In [8]:
# ==== Exercício 2 ====

np.random.seed(42)
n = 120  # número de amostras

dados = pd.DataFrame({
    "temperatura_molde": np.random.uniform(40, 100, n),     # °C
    "pressao_injecao": np.random.uniform(600, 1200, n),     # bar
    "tipo_polimero": np.random.choice(["PEAD", "PP", "PVC"], n)
})

# Codificando variável categórica (tipo_polimero)
dados = pd.get_dummies(dados, columns=["tipo_polimero"], drop_first=False)

# Criando variável alvo (tempo_resfriamento)
dados["tempo_resfriamento"] = (
    0.15 * dados["temperatura_molde"] +          # temperaturas maiores → resfriamento mais lento
    0.005 * dados["pressao_injecao"] +           # maior pressão → ligeiro aumento no tempo
    4 * dados["tipo_polimero_PEAD"] +            # PEAD tem condutividade térmica alta
    6 * dados["tipo_polimero_PP"] +              # PP demora um pouco mais
    8 * dados["tipo_polimero_PVC"] +             # PVC é o que mais demora
    np.random.normal(0, 2, n)                    # ruído aleatório
)

print(" Amostra dos dados:")
print(dados.head())

 Amostra dos dados:
   temperatura_molde  pressao_injecao  tipo_polimero_PEAD  tipo_polimero_PP  \
0          62.472407      1084.464093               False              True   
1          97.042858      1137.654780                True             False   
2          83.919637       790.802085               False              True   
3          75.919509       666.031155                True             False   
4          49.361118       736.761098               False              True   

   tipo_polimero_PVC  tempo_resfriamento  
0              False           20.154471  
1              False           23.835470  
2              False           23.084079  
3              False           23.207695  
4              False           15.512769  


In [9]:
# Separando variáveis de entrada (X) e saída (y)
X = dados.drop(columns="tempo_resfriamento")
y = dados["tempo_resfriamento"]

# Treinando e testando
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=42)

# Treinando os modelos
modelos = {
    "Regressão Linear": LinearRegression(),
    "Random Forest": RandomForestRegressor(random_state=42, n_estimators=100)
}

resultados = []

for nome, modelo in modelos.items():
    modelo.fit(X_train, y_train)
    y_pred = modelo.predict(X_test)

    # Cálculo das métricas
    eqm = mean_squared_error(y_test, y_pred)
    eam = mean_absolute_error(y_test, y_pred)
    r2 = r2_score(y_test, y_pred)

    resultados.append({
        "Modelo": nome,
        "EQM": round(eqm, 2), #obs.: 'round' arredonda o numero
        "EAM": round(eam, 2),
        "R²": round(r2, 3)
    })

resultados_df = pd.DataFrame(resultados)
print("\n Resultados de desempenho dos modelos:")
print(resultados_df)


 Resultados de desempenho dos modelos:
             Modelo   EQM   EAM     R²
0  Regressão Linear  3.85  1.59  0.645
1     Random Forest  7.00  2.19  0.355


In [10]:
# Previsão do tempo de resfriamento
novo_dado = pd.DataFrame({
    "temperatura_molde": [80],
    "pressao_injecao": [900],
    "tipo_polimero_PEAD": [0],
    "tipo_polimero_PP": [1],
    "tipo_polimero_PVC": [0]
})

# Usando o melhor modelo (Random Forest)
melhor_modelo = RandomForestRegressor(random_state=42, n_estimators=100)
melhor_modelo.fit(X_train, y_train)
previsao = melhor_modelo.predict(novo_dado)

print(f"\n Previsão de tempo de resfriamento (polímero PP): {previsao[0]:.2f} minutos")


 Previsão de tempo de resfriamento (polímero PP): 22.32 minutos
